In [282]:
import pandas as pd
import pulp
import numpy as np
import itertools

Reading in the data:

In [283]:
df = pd.read_excel("availability.xls")
df = df.replace("OK", 1)
df = df.fillna(0)

In [287]:
df.head()

,Mon 09:00 – 09:30,Mon 09:30 – 10:00,Mon 10:00 – 10:30,Mon 10:30 – 11:00,Mon 11:00 – 11:30,Mon 11:30 – 12:00,Mon 12:00 – 12:30,Mon 12:30 – 13:00,Mon 13:00 – 13:30,Mon 13:30 – 14:00,...,Fri 13:00 – 13:30,Fri 13:30 – 14:00,Fri 14:00 – 14:30,Fri 14:30 – 15:00,Fri 15:00 – 15:30,Fri 15:30 – 16:00,Fri 16:00 – 16:30,Fri 16:30 – 17:00,Fri 17:00 – 17:30,Fri 17:30 – 18:00
Rhys Lloyd - Alternative Maths,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Europa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L.A.S.T Resort,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DividedByZeroStudios,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Effervescence,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


Transform the data in to a matrix:

In [288]:
A = np.array(df)
A

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ..., 
       [0.0, 0.0, 0.0, ..., 1.0, 1.0, 1.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

Create the integer linear problem:

In [289]:
M, N = A.shape  # Dimensions
prob = pulp.LpProblem("Scheduling")
x = pulp.LpVariable.dicts("x", itertools.product(range(M), range(N)), 
                          cat=pulp.LpBinary)  # Variables

Constraints:

$$x_{ij} \leq A_{ij}$$

Ie $x_{ij}$ can be 1 iff team $i$ is available in slot $j$.

In [290]:
for index in x:  
    try:
        x[index].upBound = float(A[index])
    except ValueError:  # Seemed to be an artifact in the matrix
        x[index].upBound = 0

$$\sum_{i=1}^{M}x_{ij}\leq1$$

Ie slot $j$ can only be used by one team.

In [291]:
for slot in range(N):                                                                                                                        
    prob += sum(x[(team, slot)] for team in range(M)) <= 1 

$$\sum_{j=1}^{N}x_{ij}=1$$

Ie team $i$ uses exactly one slot.

In [292]:
for team in range(M):                                                                                                                        
    prob += sum(x[(team, slot)] for slot in range(N)) == 1 

In [293]:
prob.solve(pulp.GLPK())

1

In [294]:
pulp.LpStatus[prob.status]

'Optimal'

In [295]:
solution = []
for company in range(M):
    for slot in range(N):
        if x[(company, slot)].value() == 1:
            solution.append([df.index[company], df.columns[slot]])
df = pd.DataFrame(solution, columns=["Company", "Time slot"])

In [296]:
print(df.sort_values(by="Company"))

                           Company          Time slot
14                          Apollo  Tue 16:30 – 17:00
17                  Coding Cymraeg  Thu 13:30 – 14:00
23                       Complexus  Tue 10:00 – 10:30
3             DividedByZeroStudios  Mon 17:30 – 18:00
4                    Effervescence  Fri 17:30 – 18:00
1                           Europa  Thu 13:00 – 13:30
11                        F.E.J.L.  Tue 15:00 – 15:30
12                     Ferdie Amor  Wed 12:30 – 13:00
16                         Framtak  Tue 17:30 – 18:00
15                   FridgeVentory  Tue 17:00 – 17:30
30                   Generic Group  Tue 16:00 – 16:30
26                       GeoCampus  Tue 11:30 – 12:00
5       Green and Russian Standard  Fri 16:30 – 17:00
19                            JALE  Mon 16:00 – 16:30
27                           JEM'D  Fri 16:00 – 16:30
7                            J^2AG  Wed 16:30 – 17:00
2                   L.A.S.T Resort  Mon 12:30 – 13:00
29                MACT enter

In [297]:
df.sort_values(by="Time slot")

,Company,Time slot
9,StockSensors,Fri 11:30 – 12:00
27,JEM'D,Fri 16:00 – 16:30
5,Green and Russian Standard,Fri 16:30 – 17:00
4,Effervescence,Fri 17:30 – 18:00
2,L.A.S.T Resort,Mon 12:30 – 13:00
31,Ticket Tiger,Mon 13:00 – 13:30
20,Oakheart,Mon 15:00 – 15:30
21,MRJL,Mon 15:30 – 16:00
19,JALE,Mon 16:00 – 16:30
18,The Cyps,Mon 16:30 – 17:00
